\# This notebook has steps for model initialisation and Training**



In [1]:
!pip install transformers
!pip install tokenizer
!pip install datasets
!pip install rouge_score
!pip install sentencepiece
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 930.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=92fb7cb2bd18f6c1b4fb3633d8e7a5c631c0af101c0bba626b5899921581a662
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from google.colab import drive
import pandas as pd
import textwrap
import json
import os
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import logging
transformers_logger = logging.getLogger("transformers")
logging.getLogger("transformers").setLevel(logging.ERROR)
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset, load_metric
import torch
from rouge import Rouge


# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


**This section contains the code for loading the json file after pdf extraction and the model initialization**

In [3]:
#Load the json file after pdf extraction
dataset_path = "/content/drive/My Drive/Colab Notebooks/CS5242 Project/"

def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Join the paths
data_file_path = os.path.join(dataset_path, 'testimage16.json')

# Load  data
pdf_data = load_data(data_file_path)

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Define the Model
class SummarizationModel:
    def __init__(self, model_name, device):
        self.model_name = model_name
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name).to(DEVICE)
        self.config=BartForConditionalGeneration.from_pretrained(model_name).config

#Instantiate the model

model_name = "facebook/bart-large-cnn"
model_summarizer = SummarizationModel(model_name, device=DEVICE)
model = model_summarizer.model
tokenizer=model_summarizer.tokenizer
#print(modelsummarizer.config)

#

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

**Model Inference**

In [4]:
#Generate Summary for the content using the loaded model
def generate_summary(content):
        max_length=300
        num_beams=4
        inputs = tokenizer(content, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs.input_ids.to(DEVICE), max_length=max_length, num_beams=num_beams, early_stopping=True)
        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary_text

#Pase each sections and subsection to generate summaries from the model
def process_section(section,results):

    # Process the content of the each section
    section_summary_results = {}
    content = section["Text"]
    section_name=section["Section"]
    summary_text = generate_summary(content)
    section_summary_results["Section Name"] = section_name
    section_summary_results["Generated Summary"] = summary_text
    results.append(section_summary_results)
    print("Section Name: ", section_name)
    wrapped_output = textwrap.fill(str(summary_text), width=80)
    print("Generated Summary: ", wrapped_output)
        # Process the subsections if they exist
    if "Subsections" in section:
        for subsection in section["Subsections"]:
            process_section(subsection,results)


# Summarize the section contents and subsection contents
def summarize_pdf(pdf_data, output_file):
    all_results = []
    for section in pdf_data:
        process_section(section,all_results)
    with open(output_file, "w") as json_file:
        json.dump(all_results, json_file, indent=4)

#Write the final summary to the summary jsonfile
output_file = "summary_results_facebookbart.json"
summarize_pdf(pdf_data, output_file)


Section Name:  1 Introduction
Generated Summary:  Transformers have become the model of choice in natural language processing
(nlp) in computer vision, however, convolutional architectures remain dominant.
inspired by the transformer scaling successes in nlp, we experiment with
applying a standard transformer directly to images, with the fewest possible
modications. We find that large scale training trumps inductive bias.
Section Name:  2 Related Work
Generated Summary:  transformers were proposed by vaswani et al. for machine translation. They have
since be- come the state of the art method in many nlp tasks. We show that large
scale pre training makes vanilla transformers competitive with (or even better
than) state-of-the-art cnns.
Section Name:  3 Method
Generated Summary:  CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery.
Please submit your best shots of the U.S. for next week. Visit CNN.com/Travel
next Friday for a new gallery of snapshots from around th